In [ ]:
%pip install crewai
%pip install 'crewai[tools]'

In [ ]:
# from dotenv import load_dotenv
# load_dotenv()

import os

# os.environ["OPENAI_API_BASE"] = "http://localhost:11434/v1"
# os.environ["OPENAI_MODEL_NAME"] = "llama3:8b"
os.environ["OPENAI_API_KEY"] = "<KEY>"
os.environ["SERPER_API_KEY"] = "<KEY>"

In [ ]:
from crewai import Agent
from crewai_tools import SerperDevTool
from crewai_tools.tools import WebsiteSearchTool, FileReadTool

web_search_tool = WebsiteSearchTool()
serper_dev_tool = SerperDevTool()
file_read_tool = FileReadTool(
    file_path='job_description_example.md',
    description='A tool to read the job description example file.'
)


class Agents():
    def research_agent(self):
        return Agent(
            role='Research Analyst',
            goal='Analyze the company website and provided description to extract insights on specific needs.',
            tools=[web_search_tool, serper_dev_tool],
            backstory='Expert in analyzing company, identifying key values and needs from various sources, including websites and brief descriptions.',
            verbose=True
        )

    def writer_agent(self):
        return Agent(
            role='Job Description Writer',
            goal='Use insights from the Research Analyst to create a detailed, engaging, and enticing job posting.',
            tools=[web_search_tool, serper_dev_tool, file_read_tool],
            backstory='Skilled in crafting compelling job descriptions that resonate with the company\'s values and attract the right candidates.',
            verbose=True
        )

    def review_agent(self):
        return Agent(
            role='Review and Editing Specialist',
            goal='Review the job posting for clarity, engagement, grammatical accuracy, and alignment with company values and refine it to ensure perfection.',
            tools=[web_search_tool, serper_dev_tool, file_read_tool],
            backstory='A meticulous editor with an eye for detail, ensuring every piece of content is clear, engaging, and grammatically perfect.',
            verbose=True
        )

In [ ]:
from textwrap import dedent
from crewai import Task


class Tasks():
    def research_role_requirements_task(self, agent, hiring_needs):
        return Task(
            description=dedent(
                f"""Based on the hiring manager's needs: "{hiring_needs}", identify the key skills, experiences, and qualities the ideal candidate should possess for the role. Consider the company's current projects, its competitive landscape, and industry trends. Prepare a list of recommended job requirements and qualifications that align with the company's needs and values."""
            ),
            expected_output=dedent(
                f"""A list of recommended skills, experiences, and qualities for the ideal candidate, aligned with the company's ongoing projects, and the specific role's requirements."""
            ),
            agent=agent
        )

    def draft_job_posting_task(self, agent, company_description, hiring_needs, specific_benefits):
        return Task(
            description=dedent(
                f"""Draft a job posting for the role described by the hiring manager: "{hiring_needs}". Use the insights on "{company_description}" to start with a compelling introduction, followed by a detailed role description, responsibilities, and required skills and qualifications. Ensure the tone aligns with the company's unique benefits or opportunities offered by the company.\nSpecific benefits: "{specific_benefits}"""
            ),
            expected_output=dedent(
                f"""A detailed, engaging job posting that includes an introduction, role description, responsibilities, requirements, and unique company benefits."""
            ),
            agent=agent
        )

    def add_test_assignment_task(self, agent, test_assignment):
        return Task(
            description=dedent(
                f"""Add a test assignment to the job posting. The test assignment should be relevant to the job role and allow candidates to demonstrate their skills and problem-solving abilities.\nTest assignment: "{test_assignment}"""
            ),
            expected_output=dedent(
                f"""A test assignment section in the job posting, describing the task candidates need to complete to apply for the job."""
            ),
            agent=agent
        )

    def add_test_questions_task(self, agent, test_questions):
        return Task(
            description=dedent(
                f"""Add test questions to the job posting. These questions should help assess the candidate's suitability for the role.\nTest questions: "{test_questions}"""
            ),
            expected_output=dedent(
                f"""A test questions section in the job posting, listing the questions candidates need to answer as part of their application."""
            ),
            agent=agent
        )

    def review_and_edit_job_posting_task(self, agent, hiring_needs):
        return Task(
            description=dedent(
                f"""Review the draft job posting for the role: "{hiring_needs}". Check for clarity, engagement, grammatical accuracy. Edit and refine the content, ensuring it speaks directly to the desired candidates and accurately reflects the role's unique benefits and opportunities. Provide feedback for any necessary revisions."""
            ),
            expected_output=dedent(
                f"""A polished, error-free job posting that is clear and engaging. Feedback on potential improvements and final approval for publishing. Formatted in markdown."""
            ),
            agent=agent,
            output_file="job_posting.md"
        )

In [ ]:
from crewai import Crew

tasks = Tasks()
agents = Agents()

# company_description = input("What is the company description?\n")
# company_domain = input("What is the company domain?\n")
# hiring_needs = input("What are the hiring needs?\n")
# specific_benefits = input("What are specific_benefits you offer?\n")

# User inputs
company_description = "Ivinco Ltd."
company_domain = "Kubernetes, tech support, hi-load, PHP, Docker"
hiring_needs = "Senior Software Developer (PHP)"
specific_benefits = "Payment vacation"
test_assignment = "Build a small application using PHP and Docker."
test_questions = [
    "Describe your experience with Kubernetes.",
    "How do you handle high-load scenarios in PHP applications?",
    "Provide an example of a challenging problem you solved in a previous role."
]

# Create Agents
researcher_agent = agents.research_agent()
writer_agent = agents.writer_agent()
review_agent = agents.review_agent()

# Define Tasks for each agent
research_role_requirements_task = tasks.research_role_requirements_task(
    researcher_agent,
    hiring_needs
)
draft_job_posting_task = tasks.draft_job_posting_task(
    writer_agent,
    company_description,
    hiring_needs,
    specific_benefits
)
add_test_assignment_task = tasks.add_test_assignment_task(
    writer_agent,
    test_assignment
)
add_test_questions_task = tasks.add_test_questions_task(
    writer_agent,
    test_questions
)
review_and_edit_job_posting_task = tasks.review_and_edit_job_posting_task(
    review_agent,
    hiring_needs
)

# Instantiate the crew with a sequential process
crew = Crew(
    agents=[
        researcher_agent,
        writer_agent,
        review_agent
    ],
    tasks=[
        research_role_requirements_task,
        draft_job_posting_task,
        add_test_assignment_task,
        add_test_questions_task,
        review_and_edit_job_posting_task
    ]
)

# Kick off the process
result = crew.kickoff()

print("Job Posting Creation Process Completed.")
print("Final Job Posting:")
print(result)